# Agenda

1. Descriptors (vs. properties)
2. Iterators
     - Iterators and classes
     - Generator functions
     - Generator expressions
3. Decorators
4. Concurrency
    - Threads
    - Processes
    - Executors

In [2]:
d = {'a':1, 'b':2, 'c':3}

In [3]:
mylist = [100, 200, 300]
d[mylist] = 4

TypeError: unhashable type: 'list'

In [4]:
t = (10, 20, 30)     # tuple is immutable, thus hashable
d[t] = 100

In [6]:
t = (10, 20, [100, 200, 300])  # tuple is immutable, *BUT* contains mutable, so *NOT* hashable
d[t] = 100

TypeError: unhashable type: 'list'

In [7]:
# mutable and hashable? YES

class Person:
    def __init__(self, name):
        self.name = name
        
p = Person('Reuven')    

d[p] = 100

In [8]:
p.name

'Reuven'

In [9]:
p.name = 'something else'

In [10]:
d[p]

100

In [11]:
hash(p)

284314249

In [12]:
id(p)

4549027984

In [13]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __hash__(self):
        return hash(self.name)
        
p = Person('Reuven')    

d[p] = 100

In [14]:
d[p]

100

In [15]:
p.name = 'something else'

In [16]:
d[p]

KeyError: <__main__.Person object at 0x10f24ab30>

In [17]:
import random

class Person:
    def __init__(self, name):
        self.name = name
        
    def __hash__(self):
        return random.randint(0, 100_000)
        
p = Person('Reuven')    
d[p] = 100

In [18]:
d[p]

KeyError: <__main__.Person object at 0x10f24ae30>

In [19]:
# Properties
# looks like data, acts like getter/setter

class Thermostat:
    def __init__(self):
        self._temp = 20
        
    @property
    def temp(self):
        return self._temp
    
    @temp.setter
    def temp(self, new_temp):
        if new_temp < 10:
            raise ValueError('Too low!')
        
        if new_temp > 40:
            raise ValueError('Too high!')
        
        self._temp = new_temp

In [20]:
t = Thermostat()

t.temp

20

In [21]:
t.temp = 25
t.temp

25

In [22]:
t.temp = -10

ValueError: Too low!

In [24]:
type(t.temp)  # request the attribute temp from t, an instance

int

In [25]:
type(Thermostat.temp)  # request the attribute temp from Thermostat, the class

property

# Descriptor

Properties implement the descriptor protocol:

- If I have a class attribute,
- and I request its via the instance,
- and if the object in the class attribute has the `__get__` method,
- then the `__get__` method runs, and we get its value.

In [30]:
# Properties
# looks like data, acts like getter/setter

class SafeTemp:
    def __init__(self):
        self._temp = 20
        
    def __get__(self, instance, owner_class):
        print(f'Called SafeTemp.__get__: {instance=}, {owner_class=}')
        return self._temp
    
    def __set__(self, instance, new_value):
        print(f'Called SafeTemp.__set__: {instance=}, {new_value=}')
        self._temp = new_value

class Thermostat:
    temp = SafeTemp()   # class attribute, instance of SafeTemp
    
t = Thermostat()
t.temp  # class attribute, ask for the value via the instance
t.temp = 25   # 

Called SafeTemp.__get__: instance=<__main__.Thermostat object at 0x11b18bd30>, owner_class=<class '__main__.Thermostat'>
Called SafeTemp.__set__: instance=<__main__.Thermostat object at 0x11b18bd30>, new_value=25


In [31]:
t.temp

Called SafeTemp.__get__: instance=<__main__.Thermostat object at 0x11b18bd30>, owner_class=<class '__main__.Thermostat'>


25

In [32]:
t.temp = 20

Called SafeTemp.__set__: instance=<__main__.Thermostat object at 0x11b18bd30>, new_value=20


In [33]:
class MyClass:
    def __init__(self, x):
        self.x = x
        
    def x2(self):
        return self.x * 2
    
m = MyClass(10)
m.x2()

20

In [34]:
m.x2

<bound method MyClass.x2 of <__main__.MyClass object at 0x11b18af20>>

In [ ]:
m.x2()  #  --->  MyClass.x2(m)

In [35]:
MyClass.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.MyClass.__init__(self, x)>,
              'x2': <function __main__.MyClass.x2(self)>,
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

In [36]:
class MyClass:
    def __init__(self, x):
        self.x = x
        
    @staticmethod
    def hello():
        return 'Hello!'
    
m = MyClass(10)
m.hello()

'Hello!'

In [37]:
MyClass.hello()

'Hello!'

In [38]:
type(MyClass.hello)

function

In [39]:
type(m.hello)

function

# Iterators

In [40]:
for one_item in 'abcd':
    print(one_item)

a
b
c
d


In [41]:
for one_item in 5:
    print(one_item)

TypeError: 'int' object is not iterable

In [42]:
s = 'abcd'

# ask if the object is iterable
iter('abcd')

In [43]:
iter('abcd')

In [44]:
iter('abcd')

In [45]:
# if it is iterable, store the iterator object in i
i = iter('abcd')

In [46]:
# ask for the next item
next(i)

'a'

In [47]:
next(i)

'b'

In [48]:
next(i)

'c'

In [49]:
next(i)

'd'

In [50]:
next(i)

StopIteration: 

1. Ask for an object's iterator with `iter`
   - If not iterable, we get a `TypeError` exception
   - If iterable, we get an iterator object back
2. Run `next` on that iterator that we got back
   - If we get a `StopIteration` exception, we stop
   - Or we get an object back, the next value
3. Return to step 2   

In [57]:
class MyIterator:
    def __init__(self, data):
        print(f'\tMyIterator.__init__, {data=}')
        self.data = data
        self.index = 0
        
    def __iter__(self):    # returns the object that implements "next"
        print(f'\tMyIterator.__iter__, {self.data=}, {self.index=}')
        return self
    
    def __next__(self):
        print(f'\tMyIterator.__next__, {self.data=}, {self.index=}')
        if self.index >= len(self.data):
            print(f'\tStopping the iteration')
            raise StopIteration
            
        value = self.data[self.index]
        self.index += 1
        print(f'\tReturning {value}')
        return value

m = MyIterator('abcd')

for one_item in m:
    print(one_item)

	MyIterator.__init__, data='abcd'
	MyIterator.__iter__, self.data='abcd', self.index=0
	MyIterator.__next__, self.data='abcd', self.index=0
	Returning a
a
	MyIterator.__next__, self.data='abcd', self.index=1
	Returning b
b
	MyIterator.__next__, self.data='abcd', self.index=2
	Returning c
c
	MyIterator.__next__, self.data='abcd', self.index=3
	Returning d
d
	MyIterator.__next__, self.data='abcd', self.index=4
	Stopping the iteration


# Exercise: Circle

1. Implement a `Circle` class that is an iterator.
2. `Circle` takes two arguments:
    - `data`, a sequence (string, list, tuple)
    - `n`, number of results we'll get from our object
3. If `n` is more than the number of objects in `data`, it returns to the start

```python
c = Circle('abcd', 7)

for one_item in c:
    print(one_item)   # a b c d a b c

```

In [60]:
class Circle:
    def __init__(self, data, n):
        self.data = data
        self.n = n
        self.index = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index >= self.n:
            raise StopIteration
            
        value = self.data[self.index % len(self.data)]
        self.index += 1
        return value
    
c = Circle('abcd', 7)    

for one_item in c:
    print(one_item)

a
b
c
d
a
b
c


In [61]:
s = 'abcd'
i1 = iter(s)
i2 = iter(s)

In [62]:
i1

In [63]:
i2

In [64]:
next(i1)

'a'

In [65]:
next(i1)

'b'

In [66]:
next(i1)

'c'

In [67]:
next(i2)

'a'

In [68]:
next(i2)

'b'

In [69]:
next(i1)

'd'

In [71]:
class CircleIterator:
    def __init__(self, data, n):
        self.data = data
        self.n = n
        self.index = 0
        
    def __next__(self):
        if self.index >= self.n:
            raise StopIteration
            
        value = self.data[self.index % len(self.data)]
        self.index += 1
        return value

class Circle:
    def __init__(self, data, n):
        self.data = data
        self.n = n
        
    def __iter__(self):
        return CircleIterator(self.data, self.n)
    
    
c = Circle('abcd', 7)    

print('*** A  ***')
for one_item in c:
    print(one_item)
    
print('*** B  ***')
for one_item in c:
    print(one_item)    

*** A  ***
a
b
c
d
a
b
c
*** B  ***
a
b
c
d
a
b
c


In [72]:
class CircleIterator:
    def __init__(self, circle):
        self.circle = circle
        self.index = 0
        
    def __next__(self):
        if self.index >= self.circle.n:
            raise StopIteration
            
        value = self.circle.data[self.index % len(self.circle.data)]
        self.index += 1
        return value

class Circle:
    def __init__(self, data, n):
        self.data = data
        self.n = n
        
    def __iter__(self):
        return CircleIterator(self)
    
    
c = Circle('abcd', 7)    

print('*** A  ***')
for one_item in c:
    print(one_item)
    
print('*** B  ***')
for one_item in c:
    print(one_item)    

*** A  ***
a
b
c
d
a
b
c
*** B  ***
a
b
c
d
a
b
c


In [73]:
def myfunc():
    return 1
    return 2
    return 3

In [74]:
myfunc()

1

In [75]:
import dis
dis.dis(myfunc)

  2           0 LOAD_CONST               1 (1)
              2 RETURN_VALUE


In [76]:
def myfunc():      # generator function
    yield 1
    yield 2
    yield 3

In [77]:
myfunc()   # returns a generator

<generator object myfunc at 0x11af83b50>

In [78]:
g = myfunc()

In [79]:
next(g)   # runs the generator function through the next yield

1

In [80]:
next(g)

2

In [81]:
next(g)

3

In [82]:
next(g)

StopIteration: 

In [83]:
def myfunc():      # generator function
    yield 1
    yield 2
    yield 3

In [84]:
g = myfunc()

In [85]:
next(g)

1

In [87]:
g.gi_frame.f_lineno

2

In [88]:
next(g)

2

In [89]:
g.gi_frame.f_lineno

3

In [90]:
def fib():
    first = 0
    second = 1
    while True:
        yield first
        first, second = second, first+second

In [92]:
for one_item in fib():
    print(one_item, end=' ')
    
    if one_item > 10_000_000_000:
        break

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 165580141 267914296 433494437 701408733 1134903170 1836311903 2971215073 4807526976 7778742049 12586269025 

In [94]:
list(map(lambda x: x**2, range(10)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [95]:
def myfunc():
    yield 1
    yield 2
    return 3

g = myfunc()

In [96]:
next(g)

1

In [97]:
next(g)

2

In [98]:
next(g)

StopIteration: 3

# Exercise: `read_n`

1. Define a generator function, `read_n`:
    - First argument is a filename
    - Second argument, `n`, is an integer
2. Each iteration should return `n` lines from the file.
3. The final iteration might have fewer than `n` lines.


In [101]:
def read_n(filename, n):
    f = open(filename)
    
    while True:
        lines = []
        for i in range(n):
            lines.append(f.readline())
        
        s = ''.join(lines)
        if s:
            yield s
        else:
            break


for one_chunk in read_n('/etc/passwd', 9):
    print(one_chunk)

##
# User Database
# 
# Note that this file is consulted directly only when the system is running
# in single-user mode.  At other times this information is provided by
# Open Directory.
#
# See the opendirectoryd(8) man page for additional information about
# Open Directory.

##
nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false
root:*:0:0:System Administrator:/var/root:/bin/sh
daemon:*:1:1:System Services:/var/root:/usr/bin/false
_uucp:*:4:4:Unix to Unix Copy Protocol:/var/spool/uucp:/usr/sbin/uucico
_taskgated:*:13:13:Task Gate Daemon:/var/empty:/usr/bin/false
_networkd:*:24:24:Network Services:/var/networkd:/usr/bin/false
_installassistant:*:25:25:Install Assistant:/var/empty:/usr/bin/false
_lp:*:26:26:Printing Services:/var/spool/cups:/usr/bin/false

_postfix:*:27:27:Postfix Mail Server:/var/spool/postfix:/usr/bin/false
_scsd:*:31:31:Service Configuration Service:/var/empty:/usr/bin/false
_ces:*:32:32:Certificate Enrollment Service:/var/empty:/usr/bin/false
_appstore:*:33:

In [ ]:
def read_n(filename, n):
    f = open(filename)
    
    while True:
        s = ''.join([f.readline()
                    for i in range(n)])
        if s:
            yield s
        else:
            break


for one_chunk in read_n('/etc/passwd', 9):
    print(one_chunk)

# Next up

- Generators
    1. `itertools`
    2. Coroutines
- Decorators

Resume at 11:05

In [102]:
# Comprehensions

# list comprehension
[x**2
 for x in range(-5, 5)]

[25, 16, 9, 4, 1, 0, 1, 4, 9, 16]

In [103]:
# set comprehension
{x**2
for x in range(-5, 5)}

{0, 1, 4, 9, 16, 25}

In [104]:
# dict comprehension
{x:x**2
for x in range(-5, 5)}

{-5: 25, -4: 16, -3: 9, -2: 4, -1: 1, 0: 0, 1: 1, 2: 4, 3: 9, 4: 16}

In [106]:
# generator expression / generator comprehension
g = (x**2
 for x in range(-5,5))

g

<generator object <genexpr> at 0x11ae55380>

In [107]:
for one_item in g:
    print(one_item)

25
16
9
4
1
0
1
4
9
16


In [108]:
def myfunc(n):
    print(f'Now in myfunc({n})')
    return n

g = (myfunc(i)
    for i in range(5))

In [109]:
[myfunc(i)
    for i in range(5)]

Now in myfunc(0)
Now in myfunc(1)
Now in myfunc(2)
Now in myfunc(3)
Now in myfunc(4)


[0, 1, 2, 3, 4]

In [110]:
g = (myfunc(i)
    for i in range(5))

In [111]:
next(g)

Now in myfunc(0)


0

In [112]:
next(g)

Now in myfunc(1)


1

In [113]:
tuple(g)

Now in myfunc(2)
Now in myfunc(3)
Now in myfunc(4)


(2, 3, 4)

In [114]:
tuple(myfunc(i)
    for i in range(5))

Now in myfunc(0)
Now in myfunc(1)
Now in myfunc(2)
Now in myfunc(3)
Now in myfunc(4)


(0, 1, 2, 3, 4)

In [115]:
list('abcd')

['a', 'b', 'c', 'd']

In [116]:
tuple('abcd')

('a', 'b', 'c', 'd')

In [117]:
tuple((myfunc(i)
       for i in range(5)))

Now in myfunc(0)
Now in myfunc(1)
Now in myfunc(2)
Now in myfunc(3)
Now in myfunc(4)


(0, 1, 2, 3, 4)

In [119]:
def myfunc(n):
    # print(f'Now in myfunc({n})')
    return n

def list_from_gen():
    return list((myfunc(i)
                for i in range(5)))

def tuple_from_gen():
    return tuple((myfunc(i)
                for i in range(5)))

%timeit list_from_gen()

777 ns ± 6.68 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [120]:
%timeit tuple_from_gen()

813 ns ± 23.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [ ]:
mylist = [10, 20, 30]

'*'